In [97]:
import numpy as np
import pandas as pd
from pypfopt import EfficientFrontier,risk_models,expected_returns,objective_functions
from pypfopt.base_optimizer import BaseConvexOptimizer
from scipy.optimize import minimize
import matplotlib.pyplot as plt
import statsmodels.api as sm

In [98]:
df_info = pd.read_excel('trading-game-data-21112023.xlsx',sheet_name='info')
df_price = pd.read_excel('trading-game-data-21112023.xlsx',sheet_name='price')
df_price['Date'] = pd.to_datetime(df_price['Date'])
df_price.set_index('Date', inplace=True)
df_sp = pd.read_excel('trading-game-data-15112023.xlsx',sheet_name='index-price' )
df_sp['Date'] = pd.to_datetime(df_sp['Date'])
df_sp.set_index('Date', inplace=True)
df_size = pd.read_excel('trading-game-data-15112023.xlsx',sheet_name='size' )
df_size['Date'] = pd.to_datetime(df_size['Date'])
df_size.set_index('Date', inplace=True)
df_ptb = pd.read_excel('trading-game-data-15112023.xlsx',sheet_name='price-to-book' )
df_ptb['Date'] = pd.to_datetime(df_ptb['Date'])
df_ptb.set_index('Date', inplace=True)
df_turnover = df_size = pd.read_excel('trading-game-data-15112023.xlsx',sheet_name='turnover' )
df_turnover['Date'] = pd.to_datetime(df_turnover['Date'])
df_turnover.set_index('Date', inplace=True)
df_vix = pd.read_csv('VIX_history.csv')
df_vix['DATE'] = pd.to_datetime(df_vix['DATE'])
df_vix.set_index('DATE', inplace=True)

In [99]:
log_returns = np.log(df_price / df_price.shift(1))

In [117]:
weekly_log_returns = log_returns.resample('W-Fri').last()

In [191]:
top_max_returns = weekly_log_returns.max().sort_values(ascending = False).head(10)

In [192]:
top_std_devs = weekly_log_returns.std().sort_values(ascending = False).head(10)

In [193]:
common_indices = list(set(top_max_returns.index).intersection(set(top_std_devs.index)))

print(common_indices)

['EXPE', 'PARA', 'CMA', 'CTLT', 'FSLR', 'ZION']


In [194]:
df_tk = pd.DataFrame(columns = ['ticker','TK'])

In [195]:
def value(x,alpha,beta,loss_aversion):
    if x>=0:
        v = x**alpha
    else:
        v = -loss_aversion*(-x)**beta     
    return v

In [196]:
def prob_weighting(p,w):
    return p**w/(p**w+(1-p)**w)**(1/w)

In [197]:
for i in range(len(log_returns.columns)):
    tk = 0
    returns = log_returns.loc[:, log_returns.columns[i]].tail(25).tolist()
    positive = [return_ for return_ in returns if return_ >= 0]
    negative = [return_ for return_ in returns if return_ < 0]
    sorted_positive = sorted(positive)
    sorted_negative = sorted(negative)
    for p in range(len(sorted_positive)):
        tk = tk + value(sorted_positive[p],0.88,0.88,1.31)*(prob_weighting((p+1)/25,0.61)-prob_weighting(p/25,0.61))
    for n in range(len(sorted_negative)):
        tk = tk + + value(sorted_negative[n],0.88,0.88,1.31)*(prob_weighting((n+1)/25,0.69)-prob_weighting(n/25,0.69))
    df_tk.loc[i, 'ticker'] = log_returns.columns[i]
    df_tk.loc[i, 'TK'] = tk

In [203]:
df_tk.sort_values(by='TK',ascending=True).head(10)

,ticker,TK
355,PAYC,-0.071713
405,SEDG,-0.062679
22,ALGN,-0.047811
348,ON,-0.046766
161,ENPH,-0.044614
345,OGN,-0.039354
157,EL,-0.039259
191,FMC,-0.036946
122,CTLT,-0.036824
401,RVTY,-0.034385
